# Machine Learning Engineer Nanodegree
## Deep Learning
## Project: Build a Digit Recognition Program

In this notebook, a template is provided for you to implement your functionality in stages which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission, if necessary. Sections that begin with **'Implementation'** in the header indicate where you should begin your implementation for your project. Note that some sections of implementation are optional, and will be marked with **'Optional'** in the header.

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

----
## Step 1: Design and Test a Model Architecture
Design and implement a deep learning model that learns to recognize sequences of digits. Train the model using synthetic data generated by concatenating character images from [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) or [MNIST](http://yann.lecun.com/exdb/mnist/). To produce a synthetic sequence of digits for testing, you can for example limit yourself to sequences up to five digits, and use five classifiers on top of your deep network. You would have to incorporate an additional ‘blank’ character to account for shorter number sequences.

There are various aspects to consider when thinking about this problem:
- Your model can be derived from a deep neural net or a convolutional network.
- You could experiment sharing or not the weights between the softmax classifiers.
- You can also use a recurrent network in your deep neural net to replace the classification layers and directly emit the sequence of digits one-at-a-time.

Here is an example of a [published baseline model on this problem](http://static.googleusercontent.com/media/research.google.com/en//pubs/archive/42241.pdf). ([video](https://www.youtube.com/watch?v=vGPI_JvLoN0))

### Implementation
Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [3]:
### Your code implementation goes here.
### Feel free to use as many code cells as needed.

#!/usr/bin/env python2
# -*- coding: utf-8 -*-
"""
Created on Fri Jan 20 22:19:29 2017

@author: harik
"""

""""
    -------------------------------------------------------------------------------
    CNN Model Architecture for multi digit recognition implemented with TensorFlow
    -------------------------------------------------------------------------------
      inputs    [batch_size, 28, 140, 1]
      conv1     [patch=3x15, stride=1x1, padding=valid, 16 features]
      relu1     [relu]
      maxpool1  [patch=2x2, stride=2x2, padding=valid]
      conv2     [patch=4x20, stride=1x1, padding=valid, 32 features]
      relu2     [relu]
      maxpool2  [patch=2x2, stride=2x2, padding=valid]
      conv3     [patch=5x22, stride=1x1, padding=valid, 96 features]
      relu2     [relu]
      drop_out  10 %
      fc        [nodes=64]
      outputs   [y1,y2,y3,y4,y5]

"""

#%%

# program mode control

debug     = 1
idisplay  = 1
svhn_en   = 0
mnist_en  = 1
restore_session = 0

if mnist_en:
    num_steps  = 7125
    num_val    = 6000
    num_tests  = 10000
    img_width  = 140
    img_height = 28
    session_name = 'session/digit_recognizer.ckpt'
    #session_name = 'save/session.mymodel.run2/digit_recognizer.mnist.ckpt'
elif svhn_en:
    num_steps  = 60000
    num_val    = 5684
    num_tests  = 13068
    img_width  = 32
    img_height = 32
    session_name = 'save/session.mymodel.run2/digit_recognizer.svhn.ckpt'


import pickle
import random
import numpy as np
import tensorflow as tf

if idisplay:
    import matplotlib.pyplot as plt
    from PIL import Image
    from IPython.display import display

if mnist_en:
    print 'Loading MNIST pickled data...'
    pickle_file = 'mnist_merged/MNIST.merged.pickle'
    
    with open(pickle_file, 'rb') as f:
        save = pickle.load(f)
        X_train_samples = save['m_train_samples']
        y_train_samples = save['m_train_labels']
        X_val_samples   = save['m_val_samples']
        y_val_samples   = save['m_val_labels']
        X_test_samples  = save['m_test_samples'][:4000,]
        y_test_samples  = save['m_test_labels'][:4000,]
        del save  
        print 'Training data shape: ', X_train_samples.shape
        print 'Training label shape:', y_train_samples.shape
        print 'Validation data shape:', X_val_samples.shape
        print 'Validation label shape:', y_val_samples.shape
        print 'Test data shape:     ', X_test_samples.shape
        print 'Test label shape:    ', y_test_samples.shape
        print 'Data successfully loaded !!'
elif svhn_en:
    print 'Loading SVHN pickled data...'
    pickle_file = 'svhn_data/SVHN.pickle'

    with open(pickle_file, 'rb') as f:
        save = pickle.load(f)
        X_train_samples = save['train_dataset']
        y_train_samples = save['train_labels']
        X_val_samples   = save['valid_dataset']
        y_val_samples   = save['valid_labels']
        X_test_samples  = save['test_dataset']
        y_test_samples  = save['test_labels']
        del save  
        print 'Training data shape: ', X_train_samples.shape
        print 'Training label shape:', y_train_samples.shape
        print 'Validation data shape:', X_val_samples.shape
        print 'Validation label shape:', y_val_samples.shape
        print 'Test data shape:     ', X_test_samples.shape
        print 'Test label shape:    ', y_test_samples.shape
        print 'Data successfully loaded !!'


Loading MNIST pickled data...


IOError: [Errno 2] No such file or directory: 'mnist_merged/MNIST.merged.pickle'

In [ ]:
if idisplay:
    def display_samples(num_samples=1):
        for i in range(num_samples):        
            # train samples
            idx = random.choice(range(X_train_samples.shape[0]))
            print 'Display sample train image:', idx
            plt.imshow(X_train_samples[idx].reshape(img_height,img_width), interpolation='nearest')
            plt.show()

            # test samples
            idx = random.choice(range(X_test_samples.shape[0]))
            print 'Display sample test image:', idx
            plt.imshow(X_test_samples[idx].reshape(img_height,img_width), interpolation='nearest')
            plt.show()

    display_samples()


### Question 1
_What approach did you take in coming up with a solution to this problem?_

**Answer:** 

### Question 2
_What does your final architecture look like? (Type of model, layers, sizes, connectivity, etc.)_

**Answer:**

### Question 3
_How did you train your model? How did you generate your synthetic dataset?_ Include examples of images from the synthetic data you constructed.

**Answer:**

----
## Step 2: Train a Model on a Realistic Dataset
Once you have settled on a good architecture, you can train your model on real data. In particular, the [Street View House Numbers (SVHN)](http://ufldl.stanford.edu/housenumbers/) dataset is a good large-scale dataset collected from house numbers in Google Street View. Training on this more challenging dataset, where the digits are not neatly lined-up and have various skews, fonts and colors, likely means you have to do some hyperparameter exploration to perform well.

### Implementation
Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

### Question 4
_Describe how you set up the training and testing data for your model. How does the model perform on a realistic dataset?_

**Answer:**

### Question 5
_What changes did you have to make, if any, to achieve "good" results? Were there any options you explored that made the results worse?_

**Answer:**

### Question 6
_What were your initial and final results with testing on a realistic dataset? Do you believe your model is doing a good enough job at classifying numbers correctly?_

**Answer:**

----
## Step 3: Test a Model on Newly-Captured Images

Take several pictures of numbers that you find around you (at least five), and run them through your classifier on your computer to produce example results. Alternatively (optionally), you can try using OpenCV / SimpleCV / Pygame to capture live images from a webcam and run those through your classifier.

### Implementation
Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [ ]:


### Your code implementation goes here.
### Feel free to use as many code cells as needed.



### Question 7
_Choose five candidate images of numbers you took from around you and provide them in the report. Are there any particular qualities of the image(s) that might make classification difficult?_

**Answer:**

### Question 8
_Is your model able to perform equally well on captured pictures or a live camera stream when compared to testing on the realistic dataset?_

**Answer:**

### Optional: Question 9
_If necessary, provide documentation for how an interface was built for your model to load and classify newly-acquired images._

**Answer:** Leave blank if you did not complete this part.

----
### Step 4: Explore an Improvement for a Model

There are many things you can do once you have the basic classifier in place. One example would be to also localize where the numbers are on the image. The SVHN dataset provides bounding boxes that you can tune to train a localizer. Train a regression loss to the coordinates of the bounding box, and then test it. 

### Implementation
Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

### Question 10
_How well does your model localize numbers on the testing set from the realistic dataset? Do your classification results change at all with localization included?_

**Answer:**

### Question 11
_Test the localization function on the images you captured in **Step 3**. Does the model accurately calculate a bounding box for the numbers in the images you found? If you did not use a graphical interface, you may need to investigate the bounding boxes by hand._ Provide an example of the localization created on a captured image.

**Answer:**

----
## Optional Step 5: Build an Application or Program for a Model
Take your project one step further. If you're interested, look to build an Android application or even a more robust Python program that can interface with input images and display the classified numbers and even the bounding boxes. You can for example try to build an augmented reality app by overlaying your answer on the image like the [Word Lens](https://en.wikipedia.org/wiki/Word_Lens) app does.

Loading a TensorFlow model into a camera app on Android is demonstrated in the [TensorFlow Android demo app](https://github.com/tensorflow/tensorflow/tree/master/tensorflow/examples/android), which you can simply modify.

If you decide to explore this optional route, be sure to document your interface and implementation, along with significant results you find. You can see the additional rubric items that you could be evaluated on by [following this link](https://review.udacity.com/#!/rubrics/413/view).

### Optional Implementation
Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [ ]:


### Your optional code implementation goes here.
### Feel free to use as many code cells as needed.



### Documentation
Provide additional documentation sufficient for detailing the implementation of the Android application or Python program for visualizing the classification of numbers in images. It should be clear how the program or application works. Demonstrations should be provided. 

_Write your documentation here._

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.